In [2]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16429752517625420046
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8770341163598776130
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 1588630576309758160
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 31595870336
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15038634741862989396
physical_device_desc: "device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:00:06.0, compute capability: 7.0"
]


In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
with tf.device('/cpu:0'):
        learner= load_model("/mnt/vdb/thesis/ProtCNN.V5.set10.h5")


In [2]:
learner.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1900, 1)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 1900, 256)    512         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 1900, 256)    1024        conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 1900, 256)    0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve,auc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
def predict(_df ,condition):
    #df = _df.loc[_df.length > 30 ].copy() 
    #df = _df.loc[_df.length <= 30 ].copy()
    if condition == 0:
        df = _df.copy()
    elif condition == -1 :
        df = _df.loc[_df.length <= 30 ].copy()
    else:
        df = _df.loc[_df.length > condition ].copy() # select records with lenght > 
    #print(df.columns)
    X= np.array(df['reps'].to_list())
    X_test = np.reshape(X,(X.shape[0],X.shape[1],1))
    y_probas = learner.predict(X_test)
    threshold = 0.5
    _y = np.where(y_probas > threshold, 1, 0)
    
    df['class'] = _y
    #df.drop(columns=['reps'],inplace =True)
    accuracy = (len(df[df["class"] == 1])/len(df) )*100
    #print("predict AMP :"+str(len(df[df["class"] == 0])))
    #print("predict NonAMP:"+str(len(df[df["class"] == 1])))
    #print("predict NonAMP (in percentage):"+str(accuracy))
    return len(df[df["class"] == 1])

In [5]:
def predict_DNN(_df ,condition):
    #df = _df.loc[_df.length > 30 ].copy() 
    #df = _df.loc[_df.length <= 30 ].copy()
    if condition == 0:
        df = _df.copy()
    elif condition == -1 :
        df = _df.loc[_df.length <= 30 ].copy()
    else:
        df = _df.loc[_df.length > condition ].copy() # select records with lenght > 
    #print(df.columns)
    X= np.array(df['reps'].to_list())
    X_test = np.reshape(X,(X.shape[0],X.shape[1],1))
    y_probas = learner.predict(X_test)
    threshold = 0.5
    _y = np.where(y_probas > threshold, 1, 0)
    
    df['class'] = _y
    #df.drop(columns=['reps'],inplace =True)
    accuracy = (len(df[df["class"] == 1])/len(df) )*100
    #print("predict AMP :"+str(len(df[df["class"] == 0])))
    #print("predict NonAMP:"+str(len(df[df["class"] == 1])))
    #print("predict NonAMP (in percentage):"+str(accuracy))
    return len(df[df["class"] == 1])

In [ ]:
# summarize model.
learner.summary()

In [6]:

df = pd.read_pickle("../datasets/truthset/AMPs_truthset.reps.plk")


## Chanage format

In [7]:
X= np.array(df['reps'].to_list())
X_test = np.reshape(X,(X.shape[0],X.shape[1],1))

## Predict
To get prediction on a new dataframe, you can use the test_d

In [8]:
y_probas = learner.predict(X_test)
threshold = 0.5
_y = np.where(y_probas > threshold, 1, 0)

In [9]:
df['class'] = _y

In [10]:
len(df[df["class"] == 0]) 

50

In [12]:
len(df[df["class"] == 0]) 

58

In [13]:
len(df[df["class"] == 1] )

12

In [13]:
### PSSM

In [20]:
df = pd.read_pickle("../datasets/truthset/pssm/AMPs.truthset.reps.pkl")
X= np.array(df['reps'].to_list())
X_test = np.reshape(X,(X.shape[0],X.shape[1],1))
y_probas = learner.predict(X_test)
threshold = 0.5
_y = np.where(y_probas > threshold, 1, 0)
df['class'] = _y
len(df[df["class"] == 0]) 

32

# Predict on Shuffled sequences

In [14]:
result_df = pd.DataFrame( columns=["no_con",">200",">100",">70",">50",">30","30<="],
    index = ['shuff1','shuff2', 'shuff3',"shuff4","shuff5","shuff6","shuff7","shuff8","shuff9","shuff10"])
result_df

,no_con,>200,>100,>70,>50,>30,30<=
shuff1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shuff2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shuff3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shuff4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shuff5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shuff6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shuff7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shuff8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shuff9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shuff10,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## ProtCNN bestmodel

In [15]:
# shuff1
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_42.reps.plk")
result_df.loc['shuff1', 'no_con']=predict(df_,0)
result_df.loc['shuff1', '>200']= predict(df_,200)
result_df.loc['shuff1', '>100']=predict(df_,100)
result_df.loc['shuff1', '>70']=predict(df_,70)
result_df.loc['shuff1', '>50']=predict(df_,50)
result_df.loc['shuff1', '>30']=predict(df_,30)
result_df.loc['shuff1', '30<=']=predict(df_,-1)

In [16]:
# shuff2
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_43.reps.plk")
result_df.loc['shuff2', 'no_con']=predict(df_,0)
result_df.loc['shuff2', '>200']= predict(df_,200)
result_df.loc['shuff2', '>100']=predict(df_,100)
result_df.loc['shuff2', '>70']=predict(df_,70)
result_df.loc['shuff2', '>50']=predict(df_,50)
result_df.loc['shuff2', '>30']=predict(df_,30)
result_df.loc['shuff2', '30<=']=predict(df_,-1)

In [17]:
# shuff3
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_44.reps.plk")
result_df.loc['shuff3', 'no_con']=predict(df_,0)
result_df.loc['shuff3', '>200']= predict(df_,200)
result_df.loc['shuff3', '>100']=predict(df_,100)
result_df.loc['shuff3', '>70']=predict(df_,70)
result_df.loc['shuff3', '>50']=predict(df_,50)
result_df.loc['shuff3', '>30']=predict(df_,30)
result_df.loc['shuff3', '30<=']=predict(df_,-1)

In [18]:
# shuff4
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_45.reps.plk")
index = "shuff4"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)

In [19]:
# shuff5
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_46.reps.plk")
index = "shuff5"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)

In [20]:
# shuff6
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_47.reps.plk")
index = "shuff6"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)

In [21]:
# shuff7
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_48.reps.plk")
index = "shuff7"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)

In [22]:
# shuff8
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_49.reps.plk")
index = "shuff8"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)

In [23]:
# shuff9
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_50.reps.plk")
index = "shuff9"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)

In [24]:
# shuff10
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_51.reps.plk")
index = "shuff10"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)

In [25]:
result_df

,no_con,>200,>100,>70,>50,>30,30<=
shuff1,25,3,4,11,14,20,5
shuff2,34,2,2,9,20,26,8
shuff3,26,4,4,10,13,18,8
shuff4,22,4,6,10,12,16,6
shuff5,27,4,6,11,13,19,8
shuff6,33,3,4,8,16,24,9
shuff7,25,3,5,10,15,19,6
shuff8,26,2,3,10,13,18,8
shuff9,37,4,5,10,23,28,9
shuff10,24,3,5,9,10,18,6


## CNN input 1900 + train on  short 30 AAs

In [8]:
learner.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1900, 1)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1900, 1900)        7600      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 950, 1900)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1805000)           0         
_________________________________________________________________
dense (Dense)                (None, 128)               231040128 
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)               

In [3]:
deployed_path = "/mnt/vdb/thesis/best_CNN.1900._30.testagin.hdf5" #"/mnt/vdb/thesis/best_CNN.1900._30.hdf5" 
#deployed_path = "TubularLearner.fastAI._stage2.pth"
learner = load_model(deployed_path)
def predict_DNN(_df ,condition):
    #df = _df.loc[_df.length > 30 ].copy() 
    #df = _df.loc[_df.length <= 30 ].copy()
    if condition == 0:
        df = _df.copy()
    elif condition == -1 :
        df = _df.loc[_df.length <= 30 ].copy()
    else:
        df = _df.loc[_df.length > condition ].copy() # select records with lenght > 
    #print(df.columns)
    X= np.array(df['reps'].to_list())
    X_test = np.reshape(X,(X.shape[0],X.shape[1],1))
    y_probas = learner.predict(X_test)
    threshold = 0.5
    _y = np.where(y_probas > threshold, 1, 0)
    
    df['class'] = _y
    #df.drop(columns=['reps'],inplace =True)
    accuracy = (len(df[df["class"] == 1])/len(df) )*100
    #print("predict AMP :"+str(len(df[df["class"] == 0])))
    #print("predict NonAMP:"+str(len(df[df["class"] == 1])))
    #print("predict NonAMP (in percentage):"+str(accuracy))
    return len(df[df["class"] == 1])

learner.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1900, 1)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1900, 1900)        7600      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 950, 1900)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1805000)           0         
_________________________________________________________________
dense (Dense)                (None, 128)               231040128 
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)               

In [4]:
result_df = pd.DataFrame( columns=["no_con",">200",">100",">70",">50",">30","30<="],
    index = ['shuff1','shuff2', 'shuff3',"shuff4","shuff5","shuff6","shuff7","shuff8","shuff9","shuff10"])
# shuff1
index = "shuff1"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_42.reps.plk")
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff2
index = "shuff2"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_43.reps.plk")
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff3
index = "shuff3"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_44.reps.plk")
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff4
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_45.reps.plk")
index = "shuff4"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff5
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_46.reps.plk")
index = "shuff5"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff6
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_47.reps.plk")
index = "shuff6"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff7
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_48.reps.plk")
index = "shuff7"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff8
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_49.reps.plk")
index = "shuff8"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff9
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_50.reps.plk")
index = "shuff9"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff10
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_51.reps.plk")
index = "shuff10"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)

result_df

,no_con,>200,>100,>70,>50,>30,30<=
shuff1,15,4,7,10,10,12,3
shuff2,23,4,7,16,18,22,1
shuff3,22,4,7,10,13,18,4
shuff4,17,4,7,8,8,11,6
shuff5,19,4,6,8,8,13,6
shuff6,28,4,7,13,18,24,4
shuff7,31,4,7,9,18,24,7
shuff8,21,4,7,9,10,16,5
shuff9,17,4,6,9,13,14,3
shuff10,20,4,7,11,12,15,5


## CNN from Deep-AmPEP30 short 30 AAs

In [5]:
deployed_path = "/mnt/vdb/thesis/best_CNN._30.hdf5"
#deployed_path = "TubularLearner.fastAI._stage2.pth"
learner = load_model(deployed_path)

In [7]:
result_df = pd.DataFrame( columns=["no_con",">200",">100",">70",">50",">30","30<="],
    index = ['shuff1','shuff2', 'shuff3',"shuff4","shuff5","shuff6","shuff7","shuff8","shuff9","shuff10"])
# shuff1
index = "shuff1"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_42.reps.plk")
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff2
index = "shuff2"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_43.reps.plk")
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff3
index = "shuff3"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_44.reps.plk")
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff4
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_45.reps.plk")
index = "shuff4"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff5
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_46.reps.plk")
index = "shuff5"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff6
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_47.reps.plk")
index = "shuff6"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff7
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_48.reps.plk")
index = "shuff7"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff8
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_49.reps.plk")
index = "shuff8"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff9
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_50.reps.plk")
index = "shuff9"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff10
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_51.reps.plk")
index = "shuff10"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)

result_df

,no_con,>200,>100,>70,>50,>30,30<=
shuff1,15,0,3,5,8,12,3
shuff2,26,0,3,12,19,23,3
shuff3,23,0,3,11,14,20,3
shuff4,10,0,3,4,4,6,4
shuff5,13,1,2,5,5,12,1
shuff6,23,1,4,11,16,18,5
shuff7,27,0,3,5,16,21,6
shuff8,9,0,3,4,5,6,3
shuff9,23,0,3,8,21,22,1
shuff10,17,0,3,7,8,10,7


In [3]:
## Retrain 256
deployed_path = "/mnt/vdb/thesis/AmPPEP30.1900.hdf5"
#deployed_path = "TubularLearner.fastAI._stage2.pth"
learner = load_model(deployed_path)
result_df = pd.DataFrame( columns=["no_con",">200",">100",">70",">50",">30","30<="],
    index = ['shuff1','shuff2', 'shuff3',"shuff4","shuff5","shuff6","shuff7","shuff8","shuff9","shuff10"])
# shuff1
index = "shuff1"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_42.reps.plk")
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff2
index = "shuff2"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_43.reps.plk")
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff3
index = "shuff3"
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_44.reps.plk")
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff4
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_45.reps.plk")
index = "shuff4"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff5
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_46.reps.plk")
index = "shuff5"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff6
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_47.reps.plk")
index = "shuff6"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff7
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_48.reps.plk")
index = "shuff7"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff8
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_49.reps.plk")
index = "shuff8"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff9
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_50.reps.plk")
index = "shuff9"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)
# shuff10
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_51.reps.plk")
index = "shuff10"
result_df.loc[index, 'no_con']=predict_DNN(df_,0)
result_df.loc[index, '>200']= predict_DNN(df_,200)
result_df.loc[index, '>100']=predict_DNN(df_,100)
result_df.loc[index, '>70']=predict_DNN(df_,70)
result_df.loc[index, '>50']=predict_DNN(df_,50)
result_df.loc[index, '>30']=predict_DNN(df_,30)
result_df.loc[index, '30<=']=predict_DNN(df_,-1)

result_df

ValueError: Unknown layer: Functional